In [ ]:
#pysparkに必要なライブラリを読み込む
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

#spark sessionの作成
# spark.ui.enabled trueとするとSparkのGUI画面を確認することができます
# spark.eventLog.enabled true　とすると　GUIで実行ログを確認することができます
# GUIなどの確認は次のチャプターで説明を行います。
spark = SparkSession.builder \
    .appName("chapter2") \
    .config("hive.exec.dynamic.partition", "true") \
    .config("hive.exec.dynamic.partition.mode", "nonstrict") \
    .config("spark.sql.session.timeZone", "JST") \
    .config("spark.ui.enabled","true") \
    .config("spark.eventLog.enabled","true") \
    .enableHiveSupport() \
    .getOrCreate()

# Spark Sessionとは？

Javaで言うところのインスタンスを作る作業のことです(new Class())。  
今回の場合は、アプリケーション名が「chapter2」で作成を行っています。  

configの部分で  
非常に細かい設定ができるので、詳しくは公式のドキュメントを参考にしてください https://spark.apache.org/docs/3.1.1/  
一部メモリの設定いついては「Sparkを本番環境で動かす」チャプターにて紹介します

# Sparkを用いたバッチにおけるデータエンジニアリング一連の流れ

1. データソースの読み込み(今回は、人口統計データ(/dataset/jinkou.csv))　ETL(Extract Transform Load)で言うEの部分
2. 変換を行う(集計等を行う)　DataFrame処理 or SQL処理の２パターンで実行可能 ETL(Extract Transform Load)で言うTの部分
3. カラムナーフォーマットへ変換する ETL(Extract Transform Load)で言うTの部分
4. 出力したデータをみんなに見やすくするため(BIツールから参照できるように)テーブルを作成する ETL(Extract Transform Load)で言うLの部分

よくある、関数の羅列をするのではなく、実業務に沿った形で流れを紹介していきます。

## データソース
データの源。リレーショナルデータベースのときもあれば、今回のようにファイルの形式のときもある。  
更に進むと、PDFやEXCELなんて事もあります。  
ストリーミングだとIoTであったり、Webブラウザのアクセスログだったりとデータになりうるものは無限に存在しています。  

## 変換処理
ETL（Extract Transform Load）というと少し定義として広いのかもしれないのですが、  
データを整形してより分析向けの形(フォーマット変換や圧縮含む)にしたり、精度の高いデータを作成する行為のことです。  
そのため、ETLというとバッチ処理のイメージを持つ人も多いかもしれませんが、ストリーミングデータにも適用される言葉です。   

Lの処理はSparkにおいてDataFrameもしくはSQLで処理することができる(RDDと呼ばれるものもあるが、労力の割に実際は出番はあまりなく今回は取り扱わない)

## カラムナーフォーマットへ変換を行う
ビッグデータの世界では、Apache Parquet と呼ばれるフォーマットが広く使われています。  
CSV形式のようなローフォーマットはビックデータ処理において処理効率が悪いため、早い段階でParquetに変換を行います。  
分析用のSQLの実行であったり、複数台で処理することに向いているフォーマットです。  

Parquetの特徴としては以下になります。

- カラムナー（ストレージ）フォーマット
- カラムごとに圧縮が効くため、効率よくデータをストアできる
- 多くのプロダクトがサポートしている

多くのプロダクトはParquetを取り込んだり処理したりする機能を提供してくれており単体ではなく総合で使えるフォーマットです。

## テーブル形式での保存

多くは、実データとテーブル定義が分離された`ロケーション方式`をとっている。  
後ほど実際に作成してみますが、イメージは以下のような感じです。

```
CREATE EXTERNAL TABLE IF NOT EXISTS sample.sampletable ( id INT, date STRING)
PARTITIONED BY (dt INT)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
LOCATION '/Users/yuki/pyspark_batch/dataset/parquet/';

#S3などであれば、以下のように設定を変えることも可能です。
LOCATION 's3://data.platform/sample.db/raw_zone/sampletable/';


```

テーブルとして保存することによって、非エンジニアにも扱いやすくしてデータを提供することが可能です。

In [ ]:
#データソースの読み込み
#sep='\t'とすればtsvでも読み込みが可能です
#multiLineは、CSVやTSVの各カラムに改行が含まれていた時の対策です。
df=spark.read.option("multiLine", "true").option("encoding", "SJIS").csv("/Users/yuki/pyspark_batch/dataset/jinko.csv", header=True, sep=',', inferSchema=False)
df.count()

In [ ]:
df.show(truncate=False)

In [ ]:
#大正や昭和、平成はもう不要かなと感じたら変換処理にて
df_after_t=df.where(df."和暦（年）"== "令和")

In [103]:
#うーん使いづらい。。(日本語))
from pyspark.sql.types import LongType, StructType, StructField, StringType
from pyspark.sql.functions import col

#スキーマ設定をしていきましょう
# カラム名、型、デフォルト値で設定していきます
struct = StructType([
    StructField("code", StringType(), False),
    StructField("kenmei", StringType(), False),
    StructField("gengo", StringType(), False),
    StructField("wareki", StringType(), False),
    StructField("seireki", StringType(), False),
    StructField("chu", StringType(), False),
    StructField("jinko_male", StringType(), False),
    StructField("jinko_female", StringType(), False)
])
df=spark.read.option("multiLine", "true").option("encoding", "SJIS").csv("/Users/yuki/pyspark_batch/dataset/jinko.csv", header=False, sep=',', inferSchema=False,schema=struct)
df.show()

+--------------+----------+-----+----------+----------+----+------------+------------+
|          code|    kenmei|gengo|    wareki|   seireki| chu|  jinko_male|jinko_female|
+--------------+----------+-----+----------+----------+----+------------+------------+
|都道府県コード|都道府県名| 元号|和暦（年）|西暦（年）|  注|人口（総数）|  人口（男）|
|            00|      全国| 大正|         9|      1920|null|    55963053|    28044185|
|            01|    北海道| 大正|         9|      1920|null|     2359183|     1244322|
|            02|    青森県| 大正|         9|      1920|null|      756454|      381293|
|            03|    岩手県| 大正|         9|      1920|null|      845540|      421069|
|            04|    宮城県| 大正|         9|      1920|null|      961768|      485309|
|            05|    秋田県| 大正|         9|      1920|null|      898537|      453682|
|            06|    山形県| 大正|         9|      1920|null|      968925|      478328|
|            07|    福島県| 大正|         9|      1920|null|     1362750|      673525|
|            08|    茨城県| 大正|   

In [104]:
# 大正や昭はもう不要かなと感じたら変換処理にて
# ヘッダーも同時に除外してしまう
df.where(df.gengo == "平成").count()
# where 以外にもfilterと呼ばれるものがあります。機能は同じなので好きな方を選んで大丈夫です
df.filter(df.gengo == "平成").count()

300

In [125]:
#集計をしてみます
#平成の県ごとの男女の数の平均
import pyspark.sql.functions as sf
df.where(df.gengo == "平成").groupBy("kenmei").agg(sf.avg("jinko_male").alias("male_avg"),sf.avg("jinko_female").alias("female_avg")).show()

#「人口集中地区以外の地区」が邪魔ですね。

+----------------------+--------------------+--------------------+
|                kenmei|            male_avg|          female_avg|
+----------------------+--------------------+--------------------+
|人口集中地区以外の地区|4.3248248333333336E7|2.0976203166666668E7|
|                佐賀県|            864635.0|            408192.5|
|                栃木県|  1987157.3333333333|   987741.8333333334|
|                京都府|  2628424.6666666665|  1268325.3333333333|
|                香川県|           1009635.5|   485871.8333333333|
|                愛媛県|           1466564.5|   692188.3333333334|
|                秋田県|  1147506.8333333333|   542928.3333333334|
|                広島県|  2865315.3333333335|           1387308.5|
|                宮崎県|           1151179.5|            542386.5|
|              鹿児島県|           1747640.0|            818506.0|
|                埼玉県|   6936328.166666667|  3492880.3333333335|
|                三重県|  1838127.1666666667|            893167.5|
|                島根県|   744656.166666666

In [126]:
#良さそうです！
df.where(df.gengo == "平成").groupBy("kenmei").agg(sf.avg("jinko_male").alias("male_avg"),sf.avg("jinko_female").alias("female_avg")).filter(df.kenmei != "人口集中地区以外の地区").sort("male_avg").show()

+--------+------------------+-----------------+
|  kenmei|          male_avg|       female_avg|
+--------+------------------+-----------------+
|  鳥取県| 602176.6666666666|287885.3333333333|
|  島根県| 744656.1666666666|         356034.5|
|  高知県| 790785.1666666666|372268.1666666667|
|  徳島県| 806551.1666666666|383399.1666666667|
|  福井県| 815695.1666666666|         395512.5|
|  佐賀県|          864635.0|         408192.5|
|  山梨県|          867609.0|425777.8333333333|
|  香川県|         1009635.5|485871.8333333333|
|和歌山県|1037736.3333333334|         490624.0|
|  富山県|1105906.8333333333|         532857.0|
|  秋田県|1147506.8333333333|542928.3333333334|
|  宮崎県|         1151179.5|         542386.5|
|  石川県|         1170582.5|         566064.0|
|  大分県|1210304.3333333333|571530.6666666666|
|  山形県|1211415.1666666667|         583603.5|
|  沖縄県|1333672.6666666667|         654622.0|
|  滋賀県|         1342717.0|         662391.0|
|  岩手県|1374565.8333333333|659592.6666666666|
|  奈良県|1405915.3333333333|671178.6666666666|
| 

In [128]:
#結果を一度保存しておきます
df_after_t=df.where(df.gengo == "平成").groupBy("kenmei").agg(sf.avg("jinko_male").alias("male_avg"),sf.avg("jinko_female").alias("female_avg")).filter(df.kenmei != "人口集中地区以外の地区").sort("male_avg")

# DIKWモデル
少し脇道にそれるのですが、上記の作業はDIKWモデルというものに沿った動きでです。

DIKWモデルでは、データのステージを「Data」「Infromation」「Knowledge」「Wisdom」として定義しています。

- Data(データ)
- Information（情報）
- Knowledge（知識）
- Wisdom（知恵）

これらの頭文字をとってDIKWモデルと呼ばれています。



In [ ]:
# Spark利用の停止
spark.stop()
spark.sparkContext.stop()